 <h3>The goal of this project is finding the duplicate question on Quora webiste.
 the data  used here is provided by kaggle. lets take a look at the train set  </h3>


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [21]:

df = pd.read_csv("data/train.csv")
df.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


<h3>the dataframe cosists of 6 columns:</h3>

<p> id: row/question id </p>
<p>qid1, qid2: The unique ID of question in question1, question2 columns </p>
<p>question1, question2: questions text content </p>
<p>is_duplicate: where or not the two questions in a row means the same: this is what we are supposed to predict</p>

In [4]:
round(df['is_duplicate'].mean()*100, 2)

36.92

<h4>as we can see from mean value only about 37% of the train set questions are duplicated<h4>

In [5]:
pd.isnull(df).any()

id              False
qid1            False
qid2            False
question1       False
question2        True
is_duplicate    False
dtype: bool

<h4>And some of the rows in question2 are null</h4>

<h3>Feature analyzing:</h3>
<p>For appyling a perdication model we need to extract the features from the columns first</p>
<p>First we take a look at the test set to see what columns we have there:</p>



In [7]:
dg = pd.read_csv("data/test.csv")
dg.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


<p>The only given columns in the test set are question ids and their text so we have question1 and question2 columns to build the training features</p>

<h2>TF-IDF:</h2>
<p>TF-IDF (term frequency–inverse document frequency) is a natural language proccessing method to find how important a word in a set of documents (corpus). the first part, TF calucaltes the the number of times a term appears in a document. The second part, IDF is the inverse of the proportion of documents that the word appears in  to eliminate the weight of terms that occur very frequently in the corpus. </p>

<p>Here I am going to use sklearn's TfidfVectorizer to compute TF-IDF matrix</p>

<p>we fist combine the two question columns to make one big bag of words for TF-IDF transformation:</p>

In [8]:
first = df[['question1']]
second =  df[['question2']]
first.columns = ["question"]
second.columns = ["question"]
dfq = pd.concat([first, second], axis=0, ignore_index=True).fillna('')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features = 20, stop_words='english').fit_transform(dfq['question'].values)



In [15]:
tfidf

<808580x20 sparse matrix of type '<type 'numpy.float64'>'
	with 431224 stored elements in Compressed Sparse Row format>

<p>one way of extracting features from TF_IDF transformation is substracting the two question columns matrices to create a measure for similarity:</p>

In [19]:
N = len(df)
X_tfidf = (np.abs(tfidf[:N] - tfidf[N:])).toarray()

<h2>XGBoost Model:</h2>
<p>for this project Im using xgboost model on train data set. we first aplit the train data to test and train sets to tune XGBoost parameters using the prediction accuracy:</p>

In [25]:
from sklearn.model_selection import train_test_split

y =  df['is_duplicate'].values

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=4242)





In [27]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import log_loss

params = {'n_estimators': 300, 'max_depth': 4, 'min_samples_split': 4,
          'learning_rate': 0.01, 'loss': 'ls'}
xgboost = GradientBoostingRegressor(**params)
xgboost.fit(X_train, y_train)



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=4,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [43]:
y_pred = xgboost.predict(X_test)

print('Prediction score: {}' .format(round(log_loss(y_test, y_pred), 6)))

Prediction score: 0.641246


<p>by comparing the prediction score to Kaggle's leadbord scores, the prediciton ranking would be in about 82% top which means that TF-IDF method is not doing well. based on a bit of reading up on  TF-IDF, it seems like that since our corpus is consists of only two documents, the IDF is working very poorly</p>

<p>Next we are trying an alternative feature: word share. I found this useful NLP feature on the Kaggle forums, and figured it's worth a try.</p>

In [57]:
from nltk.corpus import stopwords
from __future__ import division

stops = set(stopwords.words("english"))

def word_match(row):
    words1 = set(str(row.question1).lower().split()) - stops
    words2 = set(str(row.question2).lower().split()) - stops
    shared = words1 & words2
    if(len(words1)==0 or len(words2) == 0):
        return 0
    else:
        return (2*len(shared)/(len(words1)+len(words2)))

In [58]:
X_word_match = df.apply(word_match, axis=1, raw=True, reduce=False)


In [60]:
X_WS = pd.DataFrame(X_word_match, columns=["word_shared"])
X_WS.head()

,word_shared
0,0.727273
1,0.307692
2,0.363636
3,0.000000
4,0.000000


<p>Here I'm adding this new feature to the exsting tf-idf features to see if combining them yields improved results.</p>

In [61]:
X = np.concatenate([X_tfidf, X_WS.values], axis=1)
X.shape

(404290, 21)

<p>applying XGBoost to the new set of features:</p>

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3434)


In [65]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import log_loss

xgboost.fit(X_train, y_train)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=4,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [66]:
y_pred = xgboost.predict(X_test)
print('Prediction score: {}' .format(round(log_loss(y_test, y_pred), 6)))

Prediction score: 0.462067
